# Climate Protection Actions Around the World

Looking at climate change performance index via https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

Goal: Produce interactive map that allows for comparison of climate change index across countries as well as more information about what specifically is contributing to to climate change and what needs to be altered.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Data Importing, Cleaning, and Preparation

### Loading Base Data

Base data contains the 2022 Climate Change Performance Index results from  https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

In [2]:
# Remove all rows with NaN values because all information is necessary
ccpi = pd.read_csv("data/ccpi.csv").dropna()
ccpi.head()

,rank,country,iso,score
0,1,Denmark,DNK,76.67
1,2,Sweden,SWE,74.22
2,3,Norway,NOR,73.29
3,4,United Kingdom,GBR,73.09
4,5,Morocco,MAR,71.60


### Importing Emissions Data

Emissions data contains information about different gasseous emissions ranging from 1750 to 2020 and is from https://www.kaggle.com/datasets/danielrpdias/co2-and-greenhouse-gas-emissions

In [3]:
# Drop rows where an ISO Code is not provided as they cannot be used on visualization
emissions = pd.read_csv("data/emissions.csv").dropna(subset=["iso_code"])
emissions.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [4]:
# Query only most recent available year of emissions
# Reset index and drop columns where all values are NaN
emissions_last = emissions[emissions["year"] == max(emissions["year"])].reset_index(drop=True).dropna(axis="columns", how="all")
emissions_last.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,population
0,AFG,Afghanistan,2020,12.160,NaN,0.11,0.014,NaN,0.312,NaN,...,20.150,103.833,NaN,0.01,0.01,NaN,0.01,0.02,NaN,38928341.0
1,ALB,Albania,2020,4.535,NaN,-6.76,-0.329,NaN,1.576,NaN,...,16.602,182.453,NaN,0.05,0.01,NaN,0.01,0.03,NaN,2877800.0
2,DZA,Algeria,2020,154.995,NaN,-6.99,-11.646,NaN,3.535,NaN,...,2062.656,1590.074,NaN,0.53,0.01,3.31,0.84,0.27,NaN,43851043.0
3,AND,Andorra,2020,0.466,NaN,-7.08,-0.036,NaN,6.035,NaN,...,NaN,15.071,NaN,NaN,NaN,NaN,NaN,0.00,NaN,77265.0
4,AGO,Angola,2020,22.198,NaN,-5.49,-1.289,NaN,0.675,NaN,...,40.780,324.439,NaN,0.06,NaN,1.62,0.02,0.05,NaN,32866267.0


In [5]:
list(emissions_last.columns)

['iso_code',
 'country',
 'year',
 'co2',
 'consumption_co2',
 'co2_growth_prct',
 'co2_growth_abs',
 'trade_co2',
 'co2_per_capita',
 'consumption_co2_per_capita',
 'share_global_co2',
 'cumulative_co2',
 'share_global_cumulative_co2',
 'coal_co2',
 'cement_co2',
 'flaring_co2',
 'gas_co2',
 'oil_co2',
 'other_industry_co2',
 'cement_co2_per_capita',
 'coal_co2_per_capita',
 'flaring_co2_per_capita',
 'gas_co2_per_capita',
 'oil_co2_per_capita',
 'other_co2_per_capita',
 'trade_co2_share',
 'share_global_cement_co2',
 'share_global_coal_co2',
 'share_global_flaring_co2',
 'share_global_gas_co2',
 'share_global_oil_co2',
 'share_global_other_co2',
 'cumulative_cement_co2',
 'cumulative_coal_co2',
 'cumulative_flaring_co2',
 'cumulative_gas_co2',
 'cumulative_oil_co2',
 'cumulative_other_co2',
 'share_global_cumulative_cement_co2',
 'share_global_cumulative_coal_co2',
 'share_global_cumulative_flaring_co2',
 'share_global_cumulative_gas_co2',
 'share_global_cumulative_oil_co2',
 'share_

### Feature Engineering

In [39]:
# Generate primary emissions source column
primary_source = []
primary_source_num = [] # Must include numeric representation because go.Choropleth does not allow for discrete/categorical color scales

for index, row in emissions_last.iterrows():
    sources = ["coal_co2", "gas_co2", "flaring_co2", "oil_co2", "cement_co2", "other_industry_co2"]
    
    # Generate dictionaries of values enumerations
    values = {key: row[key] for key in sources}
    nums = {key: i for i, key in enumerate(sources)}
    
    # Append data
    primary_source.append(max(values, key = values.get)[:-4].title())
    primary_source_num.append(nums.get(primary_source[-1].lower() + "_co2"))
    
emissions_last["primary_co2_source"] = primary_source
emissions_last["primary_co2_source_num"] = primary_source_num

### Joining Data Sources

In [58]:
ccpi_emissions = ccpi.loc[:, ccpi.columns != "country"].merge(right=emissions_last, how="right", left_on="iso", right_on="iso_code").drop("iso", axis=1)
ccpi_emissions.dropna(axis="columns", how="all", inplace=True)
# Remove full world from dataset to avoid scaling issues
ccpi_emissions = ccpi_emissions[(ccpi_emissions["country"] != "World")]
# Replace NaN values with "No Data" so that they can still be displayed in different hover texts
ccpi_emissions = ccpi_emissions.fillna("No Data")
ccpi_emissions.head()

,rank,score,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,...,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,population,primary_co2_source,primary_co2_source_num
0,No Data,No Data,AFG,Afghanistan,2020,12.16,No Data,0.11,0.014,No Data,...,No Data,0.01,0.01,No Data,0.01,0.02,No Data,38928341.0,Coal,0
1,No Data,No Data,ALB,Albania,2020,4.535,No Data,-6.76,-0.329,No Data,...,No Data,0.05,0.01,No Data,0.01,0.03,No Data,2877800.0,Oil,3
2,51.0,39.91,DZA,Algeria,2020,154.995,No Data,-6.99,-11.646,No Data,...,No Data,0.53,0.01,3.31,0.84,0.27,No Data,43851043.0,Gas,1
3,No Data,No Data,AND,Andorra,2020,0.466,No Data,-7.08,-0.036,No Data,...,No Data,No Data,No Data,No Data,No Data,0.0,No Data,77265.0,Coal,0
4,No Data,No Data,AGO,Angola,2020,22.198,No Data,-5.49,-1.289,No Data,...,No Data,0.06,No Data,1.62,0.02,0.05,No Data,32866267.0,Coal,0


## Plotting Data

### Setting up Plotting Variables

In [59]:
locations = ccpi_emissions["iso_code"]

hovertemplate = "<br>".join([
    "%{customdata[0]} (%{customdata[1]})", 
    "CCPI Score: %{customdata[2]}",
    "CCPI Rank: %{customdata[3]}",
    "CO2 Emissions per Capita: %{customdata[4]}", 
    "Global Share of CO2 Emissions: %{customdata[5]}",
    "Primary CO2 Emissions Source: %{customdata[6]}",
    "<extra></extra>"
])

customdata = ccpi_emissions[[
    "country", 
    "iso_code", 
    "score", 
    "rank", 
    "co2_per_capita", 
    "share_global_co2", 
    "primary_co2_source"
]]

### Generating Plot

In [60]:
fig = go.Figure(
    data=[
        go.Choropleth( 
            locations=locations, 
            z=ccpi_emissions["score"],  # Color values
            # Setting template for custom data in hover text
            hovertemplate=hovertemplate,
            colorscale="Hot",
            colorbar_title="Climate<br>Protection<br>Score",
            marker_line_width=0.1,
            # Custom data that will be included in hover text
            customdata=customdata
        ),
        # Need a new trace for each of the different data sets so that view can be properly updated
        go.Choropleth( 
            locations=locations, 
            z=ccpi_emissions["co2_per_capita"],
            hovertemplate=hovertemplate,
            colorscale="Hot",
            reversescale=True,
            colorbar_title="CO2<br>Emissions<br>per Capita",
            marker_line_width=0.1,
            customdata=customdata,
            visible=False
        ),
        go.Choropleth( 
            locations=locations, 
            z=ccpi_emissions["share_global_co2"],
            hovertemplate=hovertemplate,
            colorscale="Hot",
            reversescale=True,
            colorbar_title="Share of<br>Global CO2<br>Emissions",
            marker_line_width=0.1,
            customdata=customdata,
            visible=False
        ),
        go.Choropleth( 
            locations=locations, 
            z=ccpi_emissions["primary_co2_source_num"],
            colorscale="RdYlGn", # Color scale changes because data is categorical. 
            # Plotly does not allow for categorical go.Choropleth objects, so continuous diverging scale is used.
            colorbar=dict(
                tickvals=[0, 1, 2, 3],
                ticktext=["Coal", "Gas", "Flaring", "Oil"]
            ),
            colorbar_title="Primary CO2<br>Emissions<br>Source",
            hovertemplate=hovertemplate,
            marker_line_width=0.1,
            customdata=customdata,
            visible=False
        )
    ],
    layout=go.Layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        geo={
            "projection_type": "natural earth",
            "showcoastlines": False,
            
        },
        updatemenus=[
            # Add dropdown menu that will allow for selection of data
            dict(
                buttons=[
                    dict(
                        args=[{"visible":(True, False, False, False)}], # First trace visible, second not, third not
                        label="Climate Change Protection Index Score",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, True, False, False)}],
                        label="CO2 Emissions per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, True, False)}],
                        label="Share of Global CO2 Emissions",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, True)}],
                        label="Primary Source of CO2 Emissions",
                        method="update"
                    )
                    
                ],
                direction="down",
                pad={"r":10, "t":10},
                showactive=True,
                x=0,
                xanchor="left",
                y=1.1,
                yanchor="top"
            )
        ]
    )
)

fig.show()